In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
sns.set_theme(style="darkgrid")

In [2]:
load_df = pd.read_csv('data/load.csv')
hierarchy_df = pd.read_csv("data/hierarchy.csv")

In [3]:
def df_formatting(load_df, hierarchy_df):
    load_df = pd.melt(load_df, id_vars=["meter_id", "date"], value_vars=load_df.columns.difference(["meter_id", "date"]),
                                var_name="hour", value_name="load")
    load_df["hour"] = load_df["hour"].str.strip("h").astype(int) - 1
    load_df["timestamp"] = pd.to_datetime(load_df["date"] + " " + load_df["hour"].astype(str) + ":00:00", format="%m/%d/%Y %H:%M:%S")
    load_df["meter_id"] = load_df["meter_id"].astype(int)
    load_df = load_df.drop(columns=["date", "hour"])
    # Remove meter ids that appear in training but not in test and conversely
    aggregate_list = [('max_timestamp', 'max'), ('min_timestamp', 'min')]
    meters_df = load_df.groupby("meter_id")["timestamp"].agg(aggregate_list).reset_index()
    excluded_meters_df = meters_df[(meters_df["max_timestamp"]<dt.datetime(2011,1,1)) | (meters_df["min_timestamp"]>dt.datetime(2011,1,1))]
    excluded_meters = excluded_meters_df["meter_id"].to_list() + [236]
    load_df = load_df[~load_df["meter_id"].isin(excluded_meters)]
    data_df = load_df.merge(hierarchy_df, on="meter_id", how="left")
    return data_df

data_df = df_formatting(load_df, hierarchy_df)

In [4]:
data_df["month"] = data_df["timestamp"].dt.month
data_df["day_of_week"] = data_df["timestamp"].dt.day_name()
data_df["hour"] = data_df["timestamp"].dt.hour
data_df["year"] = data_df["timestamp"].dt.year

In [5]:
def get_datasets(df):
  aggregation_dict = {"load": "sum"}
  aggregation_dict.update({col: "first" for col in df.columns if col not in ["load", "timestamp"]})
  top_level_df = df.groupby("timestamp").agg(aggregation_dict).reset_index()
  top_level_df = top_level_df.drop(columns=["meter_id", "aggregate", "mid_level", "timestamp"])
  aggregate_level_names = pd.unique(df["aggregate"])
  aggregate_levels = {}
  for agg_level in aggregate_level_names:
    agg_level_df = df[df["aggregate"] == agg_level].copy()
    agg_level_df = agg_level_df.groupby("timestamp").agg(aggregation_dict).reset_index()
    agg_level_df = agg_level_df.drop(columns=["meter_id", "aggregate", "mid_level", "timestamp"])
    aggregate_levels[agg_level] = agg_level_df
  mid_level_names = pd.unique(df["mid_level"])
  mid_levels = {}
  for mid_level in mid_level_names:
    mid_level_df = df[df["mid_level"] == mid_level].copy()
    mid_level_df = mid_level_df.copy().groupby("timestamp").agg(aggregation_dict).reset_index()
    mid_level_df = mid_level_df.drop(columns=["meter_id", "aggregate", "mid_level", "timestamp"])
    mid_levels[mid_level] = mid_level_df
  bottom_level_names = pd.unique(df["meter_id"])
  bottom_levels = {}
  for bottom_level in bottom_level_names:
    bottom_level_df = df[df["meter_id"] == bottom_level].copy()
    bottom_level_df = bottom_level_df.copy().groupby("timestamp").agg(aggregation_dict).reset_index()
    bottom_level_df = bottom_level_df.drop(columns=["meter_id", "aggregate", "mid_level", "timestamp"])
    bottom_levels[bottom_level] = bottom_level_df
  
  return top_level_df, aggregate_levels, mid_levels, bottom_levels

In [6]:
top_level_df, aggregate_levels, mid_levels, bottom_levels = get_datasets(data_df)

In [14]:
def train_test_split(df):
    train_df = df[df["year"] < 2011]
    test_df = df[df["year"] == 2011]
    train_df = train_df.drop(columns="year")
    test_df = test_df.drop(columns="year")
    return train_df, test_df
top_level_train, top_level_test = train_test_split(top_level_df)
aggregate_train = {}
aggregate_test = {}
mid_levels_train = {}
mid_levels_test = {}
bottom_levels_train = {}
bottom_levels_test = {}
for agg in aggregate_levels.keys():
  train_df, test_df = train_test_split(aggregate_levels[agg])
  aggregate_train[agg] = train_df
  aggregate_test[agg] = test_df
for mid_level in mid_levels.keys():
  train_df, test_df = train_test_split(mid_levels[mid_level])
  mid_levels_train[mid_level] = train_df
  mid_levels_test[mid_level] = test_df
for bottom_level in bottom_levels.keys():
  train_df, test_df = train_test_split(bottom_levels[bottom_level])
  bottom_levels_train[bottom_level] = train_df
  bottom_levels_test[bottom_level] = test_df

In [13]:
meter_counts = train_df.groupby("meter_id").count()["load"].reset_index()
meter_counts = meter_counts[meter_counts["load"] < 52578]
print(meter_counts)

     meter_id   load
143       453  48955
144       454  45283
145       455  45283
146       456  40196
147       458  37268
148       468  26301
149       469  26301
150       470  25557
151       472  22102
152       477  19726
153       478  18982
154       481  15013
155       482  13895
156       491   9503
157       492   9071
158       496   7343


In [8]:
def train_historical_averages(train_df):
    historical_averages = train_df.groupby(["hour", "day_of_week", "month"]).mean().to_dict()["load"]
    return historical_averages

In [9]:
def get_average(row, historical_averages):
    return historical_averages[(row["hour"], row["day_of_week"], row["month"])]

In [10]:
def test_historical_averages(X_test, historical_averages):
    X_test["load"] = X_test.apply(lambda row: get_average(row, historical_averages), axis=1)
    return X_test["load"]

In [11]:
top_avgs = train_historical_averages(top_level_train)
agg_avgs = {}
mid_avgs = {}
bottom_avgs = {}
for agg in aggregate_train.keys():
    agg_avgs[agg] = train_historical_averages(aggregate_train[agg])

for mid in mid_levels_train.keys():
    mid_avgs[mid] = train_historical_averages(mid_levels_train[mid])

for bottom in bottom_levels_train.keys():
    bottom_avgs[bottom] = train_historical_averages(bottom_levels_train[bottom])

## Evaluate the forecasts

In [21]:
accuracies_df = pd.DataFrame(columns=["level", "id", "MAE", "RMSE"])

y_top_level_test = top_level_test["load"]
y_top_level_pred = test_historical_averages(top_level_test.drop(columns="load"), top_avgs)
accuracies_df.loc[len(accuracies_df.index)] = ["sum", "sum", mean_absolute_error(y_top_level_test, y_top_level_pred), np.sqrt(mean_squared_error(y_top_level_test, y_top_level_pred))]

for agg in aggregate_test.keys():
    y_agg_level_test = aggregate_test[agg]["load"]
    y_agg_level_pred = test_historical_averages(aggregate_test[agg].drop(columns="load"), agg_avgs[agg])
    accuracies_df.loc[len(accuracies_df.index)] = ["agg", agg, mean_absolute_error(y_agg_level_test, y_agg_level_pred), np.sqrt(mean_squared_error(y_agg_level_test, y_agg_level_pred))]

for mid in mid_levels_test.keys():
    y_mid_level_test = mid_levels_test[mid]["load"]
    y_mid_level_pred = test_historical_averages(mid_levels_test[mid].drop(columns="load"), mid_avgs[mid])
    accuracies_df.loc[len(accuracies_df.index)] = ["mid", mid, mean_absolute_error(y_mid_level_test, y_mid_level_pred), np.sqrt(mean_squared_error(y_mid_level_test, y_mid_level_pred))]

for bottom in bottom_levels_test.keys():
    try:
        y_bottom_level_test = bottom_levels_test[bottom]["load"]
        y_bottom_level_pred = test_historical_averages(bottom_levels_test[bottom].drop(columns="load"), bottom_avgs[bottom])
        accuracies_df.loc[len(accuracies_df.index)] = ["bottom", bottom, mean_absolute_error(y_bottom_level_test, y_bottom_level_pred), np.sqrt(mean_squared_error(y_bottom_level_test, y_bottom_level_pred))]
    except KeyError:
        continue

In [23]:
accuracies_df

,level,id,MAE,RMSE
0,sum,sum,127518.274559,175257.080084
1,agg,I003,22696.197586,31838.357822
2,agg,I002,108445.453600,148931.315171
3,mid,E001,5521.613433,7574.012561
4,mid,E002,15262.814643,20909.393395
...,...,...,...,...
171,bottom,478,88.123634,159.363365
172,bottom,481,1033.353186,1409.436396
173,bottom,482,340.702746,479.579097
174,bottom,491,3515.952885,4752.629684


In [24]:
accuracies_df.groupby("level")[["MAE", "RMSE"]].mean()

,MAE,RMSE
level,,
agg,65570.825593,90384.836497
bottom,1139.278802,1512.280226
mid,9587.519579,13178.393004
sum,127518.274559,175257.080084
